## Merge Reviews Data with Product Scrape Data
_By: Rachel Koenig_ 
_____

Imports

In [5]:
import json
import gzip

#Import pandas 
import pandas as pd
#Import Natual Language Toolkit
import nltk
#Import Beautiful Soup
from bs4 import BeautifulSoup   
#Import string for list of punctuation
import string
# Import the stop word list
from nltk.corpus import stopwords 
# Import Tokenizer
from nltk.tokenize import RegexpTokenizer
#Import Lemmatizer
from nltk.stem import WordNetLemmatizer
# Import stemmer.
from nltk.stem.porter import PorterStemmer

Bring in reviews dataset from:  
[WWW / SIGIR papers](http://jmcauley.ucsd.edu/data/amazon/index.html),   
R. He, J. McAuley. Modeling the visual evolution of fashion trends with one-class collaborative filtering. WWW, 2016   
J. McAuley, C. Targett, J. Shi, A. van den Hengel. Image-based recommendations on styles and substitutes. SIGIR, 2015


In [6]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

reviews = getDF('data/reviews_Clothing_Shoes_and_Jewelry_5.json.gz')

In [7]:
reviews.head()

reviewerID        asin                 reviewerName helpful  \
0  A1KLRMWW2FWPL4  0000031887  Amazon Customer "cameramom"  [0, 0]   
1  A2G5TCU2WDFZ65  0000031887              Amazon Customer  [0, 0]   
2  A1RLQXYNCMWRWN  0000031887                       Carola  [0, 0]   
3   A8U3FAMSJVHS5  0000031887                      Caromcg  [0, 0]   
4  A3GEOILWLK86XM  0000031887                           CJ  [0, 0]   

                                          reviewText  overall  \
0  This is a great tutu and at a really great pri...      5.0   
1  I bought this for my 4 yr old daughter for dan...      5.0   
2  What can I say... my daughters have it in oran...      5.0   
3  We bought several tutus at once, and they are ...      5.0   
4  Thank you Halo Heaven great product for Little...      5.0   

                         summary  unixReviewTime   reviewTime  
0  Great tutu-  not cheaply made      1297468800  02 12, 2011  
1                    Very Cute!!      1358553600  01 19, 2013  
2       I have buy more than one      1357257600   01 4, 2013  
3               Adorable, Sturdy      1398556800  04 27, 2014  
4        Grammy's Angels Love it      1394841600  03 15, 2014

In [8]:
# Check data types 
reviews.dtypes

reviewerID         object
asin               object
reviewerName       object
helpful            object
reviewText         object
overall           float64
summary            object
unixReviewTime      int64
reviewTime         object
dtype: object

In [9]:
#Convert universal time to date
reviews['review_date'] = pd.to_datetime(reviews['unixReviewTime'], unit='s')
reviews.head(3)

reviewerID        asin                 reviewerName helpful  \
0  A1KLRMWW2FWPL4  0000031887  Amazon Customer "cameramom"  [0, 0]   
1  A2G5TCU2WDFZ65  0000031887              Amazon Customer  [0, 0]   
2  A1RLQXYNCMWRWN  0000031887                       Carola  [0, 0]   

                                          reviewText  overall  \
0  This is a great tutu and at a really great pri...      5.0   
1  I bought this for my 4 yr old daughter for dan...      5.0   
2  What can I say... my daughters have it in oran...      5.0   

                         summary  unixReviewTime   reviewTime review_date  
0  Great tutu-  not cheaply made      1297468800  02 12, 2011  2011-02-12  
1                    Very Cute!!      1358553600  01 19, 2013  2013-01-19  
2       I have buy more than one      1357257600   01 4, 2013  2013-01-04

In [10]:
# Check number of rows and columns 
reviews.shape

(278677, 10)

In [11]:
#Check data types
reviews.dtypes

reviewerID                object
asin                      object
reviewerName              object
helpful                   object
reviewText                object
overall                  float64
summary                   object
unixReviewTime             int64
reviewTime                object
review_date       datetime64[ns]
dtype: object

In [12]:
#Check for nulls 
reviews.isnull().sum()

reviewerID          0
asin                0
reviewerName      452
helpful             0
reviewText          0
overall             0
summary             0
unixReviewTime      0
reviewTime          0
review_date         0
dtype: int64

In [13]:
# Check value counts of reviewerName column
reviews['reviewerName'].value_counts(dropna=False).head(20)

Amazon Customer    4001
NaN                 452
Kindle Customer     282
Jennifer            254
Sarah               245
Stephanie           243
Chris               219
Karen               193
Mary                191
Lisa                188
Michelle            185
Jessica             174
Jen                 169
Ashley              162
Amanda              158
Pen Name            158
Susan               157
Melissa             153
Kat                 152
Rachel              151
Name: reviewerName, dtype: int64

In [14]:
# Check how many unique reviewers there are in the dataset 
len(set(reviews['reviewerID']))

39387

In [15]:
# Check column names 
reviews.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime', 'review_date'],
      dtype='object')

In [16]:
reviews['helpful'].value_counts(normalize=True).head(20)
# Since 68% of the data does not have a helpful score, I do not think this column is useful.

[0, 0]    0.687735
[1, 1]    0.106668
[0, 1]    0.034883
[2, 2]    0.032389
[1, 2]    0.023382
[3, 3]    0.015218
[2, 3]    0.010187
[4, 4]    0.009053
[3, 4]    0.007726
[5, 5]    0.005756
[6, 6]    0.003890
[0, 2]    0.003660
[4, 5]    0.003592
[7, 7]    0.002828
[1, 3]    0.002551
[5, 6]    0.002304
[2, 4]    0.002264
[8, 8]    0.002175
[3, 5]    0.001995
[9, 9]    0.001669
Name: helpful, dtype: float64

In [17]:
# Drop unneeded columns 
reviews.drop(columns=['reviewerName', 'helpful', 'reviewTime'], inplace=True)

In [18]:
#Check value counts for overall column 
reviews['overall'].value_counts()

5.0    163240
4.0     58357
3.0     30425
2.0     15463
1.0     11192
Name: overall, dtype: int64

In [19]:
#Check first 5 rows of the summary column 
reviews['summary'].head()

0    Great tutu-  not cheaply made
1                      Very Cute!!
2         I have buy more than one
3                 Adorable, Sturdy
4          Grammy's Angels Love it
Name: summary, dtype: object

In [20]:
# Expand one row of the reviewText column.
reviews['reviewText'][1]

'I bought this for my 4 yr old daughter for dance class, she wore it today for the first time and the teacher thought it was adorable. I bought this to go with a light blue long sleeve leotard and was happy the colors matched up great. Price was very good too since some of these go for over $15.00 dollars.'

Read in product info csv.

In [21]:
product_info = pd.read_csv('data/final_df.csv', index_col=[0])
product_info.head()

asin                      category           color  \
0  0000031887                        Dance      unavailable   
1  0123456479   Jewelry Boxes & Organizers      unavailable   
2  1608299953                             0     unavailable   
3  1617160377                             0     unavailable   
4  B00001W0KA                             0  Buzz Lightyear   

                           demographic                  department  \
0                    Sports & Fitness           Sports & Outdoors    
1   Shoe, Jewelry & Watch Accessories   Clothing, Shoes & Jewelry    
2               Education & Reference                    Software    
3               Education & Reference                    Software    
4             Dress Up & Pretend Play                Toys & Games    

                                         description detail_type details  \
0  This fits your .     Make sure this fitsby ent...      Skirts       0   
1  This fits your .     Make sure this fitsby ent...           0       0   
2  Access for up to 5 family members Download act...           0       0   
3  Access for up to 5 family members Download act...           0       0   
4  Lead Free Child (4-6 & 7-8) Includes: Bodysuit...           0       0   

                division                                               name  \
0          Other Sports    Mystiqueshapes Girls Ballet Tutu Neon Lime Green   
1   Jewelry Accessories   SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...   
2              Languages       Learn French: Rosetta Stone French - Level 1   
3              Languages     Learn Italian: Rosetta Stone Italian - Level 1   
4               Costumes      Buzz Lightyear Boy's Deluxe Toy Story Costume   

                                                size    subcategory     type  
0                                            missing      Clothing    Girls   
1                                            missing  Jewelry Boxes        0  
2                                            missing              0        0  
3                                            missing              0        0  
4  SelectM(3T-4T)3T-4TSmall (4-6)Medium (7-8)Medi...              0        0

Check number of rows and columns.

In [22]:
product_info.shape

(13732, 13)

Check for null rows.

In [23]:
product_info.isnull().sum()

asin           0
category       0
color          0
demographic    0
department     0
description    0
detail_type    0
details        0
division       0
name           0
size           0
subcategory    0
type           0
dtype: int64

Look at column names.

In [24]:
product_info.columns

Index(['asin', 'category', 'color', 'demographic', 'department', 'description',
       'detail_type', 'details', 'division', 'name', 'size', 'subcategory',
       'type'],
      dtype='object')

Check value counts of all the rows to explore the data and get an idea of how it could be used.

In [25]:
product_info['name'].value_counts().head(10)

Simple 6MM Gemstone Round Ball Stud Earrings For Women For Teen 925 Sterling Silver 9 Birthstones More Colors    6
Birkenstock Women's Mayari Birko-Flor Sandal                                                                     5
Timex Ironman Classic 30 Mid-Size Watch                                                                          5
Hanes Men's Woven Plain-Weave Pajama Set                                                                         4
Timex Women's Indiglo Easy Reader Quartz Analog Leather Strap Watch with Date Feature                            4
Hanes Ultimate Men's 4-Pack Boxer Brief                                                                          4
Allegra K Women's Off Shoulder Self Tie Bowknot Colorblock Casual Tunic Shirt                                    4
Birkenstock Women's Gizeh Thong Sandals                                                                          4
Simple 8MM Gemstone Round Ball Stud Earrings For Women For Teen 925 Sterling Sil

In [26]:
product_info['department'].value_counts()

Clothing, Shoes & Jewelry     12876
Sports & Outdoors               569
Toys & Games                     77
Health & Household               50
Arts, Crafts & Sewing            29
Home & Kitchen                   29
Electronics                      28
Tools & Home Improvement         19
Beauty & Personal Care           18
Automotive                       11
Baby Products                     8
Office Products                   6
Cell Phones & Accessories         5
Software                          3
Industrial & Scientific           2
Purchase Circles                  1
Appliances                        1
Name: department, dtype: int64

In [27]:
product_info['department'] = product_info['department'].str.strip(' ').str.replace(',', '').str.replace('&', '').str.replace(' ', '_').str.replace('__', '_')
# Replace all commas and ampersands with and underscore and replace spaces with nothing.

In [28]:
#Check that it worked 
product_info['department'].value_counts()

Clothing_Shoes_Jewelry     12876
Sports_Outdoors              569
Toys_Games                    77
Health_Household              50
Arts_Crafts_Sewing            29
Home_Kitchen                  29
Electronics                   28
Tools_Home_Improvement        19
Beauty_Personal_Care          18
Automotive                    11
Baby_Products                  8
Office_Products                6
Cell_Phones_Accessories        5
Software                       3
Industrial_Scientific          2
Purchase_Circles               1
Appliances                     1
Name: department, dtype: int64

In [29]:
# Turn each value in a column with a 1 if the product is in that department and a 0 if not. 
departments = pd.get_dummies(product_info['department'], drop_first=True)
departments.shape

(13732, 16)

In [30]:
product_info['demographic'].value_counts().head(10)

 Women                                 7760
 Men                                   2969
 Novelty & More                         503
 Luggage & Travel Gear                  355
 Sports & Fitness                       343
 Costumes & Accessories                 271
 Girls                                  224
 Shoe, Jewelry & Watch Accessories      220
 Baby                                   205
 Outdoor Recreation                     201
Name: demographic, dtype: int64

In [31]:
product_info['demographic'] = product_info['demographic'].str.strip(' ').str.replace(',', '_').str.replace('&', '').str.replace(' ', '_').str.replace('__', '_')
# Strip white space from beginning and end 
# Replace punctuation and spaces with underscores, replace double underscore with single underscore 

In [32]:
product_info['demographic'].value_counts().head(10)

Women                             7844
Men                               2997
Novelty_More                       513
Luggage_Travel_Gear                359
Sports_Fitness                     346
Costumes_Accessories               272
Girls                              240
Shoe_Jewelry_Watch_Accessories     220
Baby                               209
Outdoor_Recreation                 202
Name: demographic, dtype: int64

In [33]:
# Turn each value in a column with a 1 if the product is in that department and a 0 if not. 
demographic = pd.get_dummies(product_info['demographic'], drop_first=True)
demographic.shape

(13732, 77)

In [34]:
product_info.head()

asin                      category           color  \
0  0000031887                        Dance      unavailable   
1  0123456479   Jewelry Boxes & Organizers      unavailable   
2  1608299953                             0     unavailable   
3  1617160377                             0     unavailable   
4  B00001W0KA                             0  Buzz Lightyear   

                      demographic              department  \
0                  Sports_Fitness         Sports_Outdoors   
1  Shoe_Jewelry_Watch_Accessories  Clothing_Shoes_Jewelry   
2             Education_Reference                Software   
3             Education_Reference                Software   
4           Dress_Up_Pretend_Play              Toys_Games   

                                         description detail_type details  \
0  This fits your .     Make sure this fitsby ent...      Skirts       0   
1  This fits your .     Make sure this fitsby ent...           0       0   
2  Access for up to 5 family members Download act...           0       0   
3  Access for up to 5 family members Download act...           0       0   
4  Lead Free Child (4-6 & 7-8) Includes: Bodysuit...           0       0   

                division                                               name  \
0          Other Sports    Mystiqueshapes Girls Ballet Tutu Neon Lime Green   
1   Jewelry Accessories   SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...   
2              Languages       Learn French: Rosetta Stone French - Level 1   
3              Languages     Learn Italian: Rosetta Stone Italian - Level 1   
4               Costumes      Buzz Lightyear Boy's Deluxe Toy Story Costume   

                                                size    subcategory     type  
0                                            missing      Clothing    Girls   
1                                            missing  Jewelry Boxes        0  
2                                            missing              0        0  
3                                            missing              0        0  
4  SelectM(3T-4T)3T-4TSmall (4-6)Medium (7-8)Medi...              0        0

In [35]:
product_info['detail_type'].value_counts().tail(20)

Sets                         1
One-Piece Pajamas            1
Boxer Briefs                 1
Gun Holsters                 1
Link                         1
Polos                        1
Tanks & Racerbacks           1
Clasp                        1
Tanks Tops                   1
Hosiery                      1
Sunglasses & Eyewear         1
Unitards                     1
Wear to Work                 1
Pajama Sets                  1
Skirts & Skorts              1
Neckties                     1
 Panties ›Briefs             1
 Gloves & Mittens ›Gloves    1
Trunks                       1
Suspenders                   1
Name: detail_type, dtype: int64

When checking out the `detail_type` value counts I noticed some additional categories that did not get split off in the first round of EDA so I did it now.

In [36]:
extra_split = product_info['detail_type'].str.split("›", expand=True)
extra_split[1].value_counts()

Baseball Caps          5
Berets                 2
Arm Warmers            2
Beanies & Knit Hats    2
Briefs                 1
Gloves                 1
Name: 1, dtype: int64

In [37]:
product_info['extra_split'] = extra_split[1]
product_info['detail_type'] = extra_split[0]

In [38]:
product_info['detail_type'] = product_info['detail_type'].str.strip(' ').str.replace(',', '_').str.replace('&', '').str.replace(' ', '_').str.replace('__', '_')
# Strip white space from beginning and end 
# Replace punctuation and spaces with underscores, replace double underscore with single underscore  

In [39]:
product_info['detail_type'].value_counts().tail(10)

Gun_Holsters        1
Sets                1
Hosiery             1
Suspenders          1
Polos               1
Tanks_Racerbacks    1
Boxer_Briefs        1
Panties             1
Trunks              1
Neckties            1
Name: detail_type, dtype: int64

In [40]:
product_info['extra_split'] = product_info['extra_split'].str.replace('&', '_').str.replace(' ', '_').str.replace('__', '_')
product_info['extra_split'].value_counts()
# Replace punctuation and spaces with underscores, replace double underscore with single underscore  

Baseball_Caps         5
Berets                2
Arm_Warmers           2
Beanies__Knit_Hats    2
Briefs                1
Gloves                1
Name: extra_split, dtype: int64

In [41]:
product_info['division'].value_counts().head(20)

 Clothing                    4004
 Shoes                       3946
 Jewelry                     1432
 Accessories                  761
 Watches                      742
 Uniforms, Work & Safety      222
0                             184
 Handbags & Wallets           182
 Exercise & Fitness           152
 Outdoor Clothing             142
 Women                        130
 Baby Girls                   121
 Travel Accessories            91
Shoes                          84
 Jewelry Accessories           84
 Baby Boys                     82
 Surf, Skate & Street          81
 Kids & Baby                   77
Clothing                       74
 Luggage                       72
Name: division, dtype: int64

In [42]:
product_info['division'] = product_info['division'].str.strip(' ').str.replace(',', '_').str.replace('&', '').str.replace(' ', '_').str.replace('__', '_')
product_info['division'].head(10)
# Strip white space from beginning and end 
# Replace punctuation and spaces with underscores, replace double underscore with single underscore  

0           Other_Sports
1    Jewelry_Accessories
2              Languages
3              Languages
4               Costumes
5              Kids_Baby
6     Travel_Accessories
7     Travel_Accessories
8     Travel_Accessories
9              Kids_Baby
Name: division, dtype: object

In [43]:
product_info['category'] = product_info['category'].str.strip(' ').str.replace(',', '_').str.replace('&', '').str.replace(' ', '_').str.replace('__', '_')
# Strip white space from beginning and end 
# Replace punctuation and spaces with underscores, replace double underscore with single underscore  

In [44]:
product_info['category'].value_counts().head(20)

Lingerie_Sleep_Lounge    896
Athletic                 839
Sandals                  754
Wrist_Watches            704
0                        636
Boots                    619
Earrings                 464
Clothing                 430
Necklaces                367
Novelty                  359
Flats                    344
Pumps                    294
Active                   292
Tops_Tees_Blouses        268
Bracelets                268
Jeans                    266
Loafers_Slip-Ons         256
Underwear                244
Fashion_Sneakers         232
Dresses                  208
Name: category, dtype: int64

In [45]:
product_info['subcategory'].value_counts().head(10)

0                     4903
 Lingerie              760
 Running               318
 Women                 305
Casual                 273
 Men                   248
Pendants               217
Platforms & Wedges     214
Stud                   194
Flip-Flops             191
Name: subcategory, dtype: int64

In [46]:
product_info['subcategory'] = product_info['subcategory'].str.strip(' ').str.replace(',', '_').str.replace('&', '').str.replace(' ', '_').str.replace('__', '_')
product_info['subcategory'].value_counts().head(10)
# Strip white space from beginning and end 
# Replace punctuation and spaces with underscores, replace double underscore with single underscore  

0                   4903
Lingerie             767
Running              454
Women                321
Casual               273
Men                  269
Pendants             217
Platforms_Wedges     214
Stud                 194
Flip-Flops           191
Name: subcategory, dtype: int64

In [47]:
product_info['type'].value_counts().head(15)

0                10994
 Bras              391
Road Running       241
 Panties           179
 Shirts            144
Sport Watches      105
 Shapewear          92
 Tops & Tees        73
 Accessories        72
Trail Running       72
 Charms             68
Pants               58
 Women              53
Robes               50
Tops                42
Name: type, dtype: int64

In [48]:
product_info['type'] = product_info['type'].str.strip(' ').str.replace(',', '_').str.replace('&', '').str.replace(' ', '_').str.replace('__', '_')
product_info['type'].value_counts().head(10)

0                10994
Bras               399
Road_Running       241
Panties            199
Shirts             147
Sport_Watches      105
Accessories         97
Shapewear           95
Charms              76
Tops_Tees           74
Name: type, dtype: int64

In [49]:
product_info.head()

asin                  category           color  \
0  0000031887                     Dance     unavailable   
1  0123456479  Jewelry_Boxes_Organizers     unavailable   
2  1608299953                         0     unavailable   
3  1617160377                         0     unavailable   
4  B00001W0KA                         0  Buzz Lightyear   

                      demographic              department  \
0                  Sports_Fitness         Sports_Outdoors   
1  Shoe_Jewelry_Watch_Accessories  Clothing_Shoes_Jewelry   
2             Education_Reference                Software   
3             Education_Reference                Software   
4           Dress_Up_Pretend_Play              Toys_Games   

                                         description detail_type details  \
0  This fits your .     Make sure this fitsby ent...      Skirts       0   
1  This fits your .     Make sure this fitsby ent...           0       0   
2  Access for up to 5 family members Download act...           0       0   
3  Access for up to 5 family members Download act...           0       0   
4  Lead Free Child (4-6 & 7-8) Includes: Bodysuit...           0       0   

              division                                               name  \
0         Other_Sports   Mystiqueshapes Girls Ballet Tutu Neon Lime Green   
1  Jewelry_Accessories  SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...   
2            Languages       Learn French: Rosetta Stone French - Level 1   
3            Languages     Learn Italian: Rosetta Stone Italian - Level 1   
4             Costumes      Buzz Lightyear Boy's Deluxe Toy Story Costume   

                                                size    subcategory   type  \
0                                            missing       Clothing  Girls   
1                                            missing  Jewelry_Boxes      0   
2                                            missing              0      0   
3                                            missing              0      0   
4  SelectM(3T-4T)3T-4TSmall (4-6)Medium (7-8)Medi...              0      0   

  extra_split  
0        None  
1        None  
2        None  
3        None  
4        None

Use `get_dummies` on the `department`, `demographic`, `detail_type`, `category`, `division`, `subcategory`, `type` and `extra_split` columns to turn each word in each row into their own columns. If the row contained that word, it will have a 1 in the column on the corresponding row, else it will have a 0.  Save each dummied column into its own DataFrame.

In [50]:
details = pd.get_dummies(product_info['detail_type'], drop_first=True)
details.head()

Adhesive_Bras  Baby_Dolls  Bandanas  Bead  Belt_Buckles  Belts  Bikinis  \
0              0           0         0     0             0      0        0   
1              0           0         0     0             0      0        0   
2              0           0         0     0             0      0        0   
3              0           0         0     0             0      0        0   
4              0           0         0     0             0      0        0   

   Bodysuits  Boxer_Briefs  Boy_Shorts  ...  Tanks_Sleeveless_Shirts  \
0          0             0           0  ...                        0   
1          0             0           0  ...                        0   
2          0             0           0  ...                        0   
3          0             0           0  ...                        0   
4          0             0           0  ...                        0   

   Tanks_Tops  Thigh_Slimmers  Tights  Tops  Trunks  Unitards  Waist_Cinchers  \
0           0               0       0     0       0         0               0   
1           0               0       0     0       0         0               0   
2           0               0       0     0       0         0               0   
3           0               0       0     0       0         0               0   
4           0               0       0     0       0         0               0   

   Wallets  Wear_to_Work  
0        0             0  
1        0             0  
2        0             0  
3        0             0  
4        0             0  

[5 rows x 64 columns]

In [51]:
category = pd.get_dummies(product_info['category'], drop_first=True)

In [52]:
division = pd.get_dummies(product_info['division'], drop_first=True)

In [53]:
subcategory = pd.get_dummies(product_info['subcategory'], drop_first=True)

In [54]:
subcategory.head()

Accessories  Accessory_Sets  Active  Active_Base_Layers  Active_Hoodies  \
0            0               0       0                   0               0   
1            0               0       0                   0               0   
2            0               0       0                   0               0   
3            0               0       0                   0               0   
4            0               0       0                   0               0   

   Active_Leggings  Active_Pants  Active_Performance  Active_Shirts_Tees  \
0                0             0                   0                   0   
1                0             0                   0                   0   
2                0             0                   0                   0   
3                0             0                   0                   0   
4                0             0                   0                   0   

   Active_Shorts  ...  Wool_Blends  Wool_Pea_Coats  Work  Work_Safety  \
0              0  ...            0               0     0            0   
1              0  ...            0               0     0            0   
2              0  ...            0               0     0            0   
3              0  ...            0               0     0            0   
4              0  ...            0               0     0            0   

   Work_Utility_Safety  Wrap  Wraps_Pashminas  Wrestling  Wrist_Watches  \
0                    0     0                0          0              0   
1                    0     0                0          0              0   
2                    0     0                0          0              0   
3                    0     0                0          0              0   
4                    0     0                0          0              0   

   Y-Necklaces  
0            0  
1            0  
2            0  
3            0  
4            0  

[5 rows x 355 columns]

In [55]:
extra_dummy = pd.get_dummies(product_info['extra_split'].fillna(0), drop_first=True)

In [56]:
extra_dummy.shape

(13732, 6)

In [57]:
# Make a list of DataFrames including the oringal product info and all the dummies 
dummies = [product_info, departments, demographic, details, category, division, subcategory, extra_dummy]


In [58]:
# Concatonate the dummy DataFrames onto the product info df vertically 
# and drop all the columns that were turned into dummies.
product_dummies = pd.concat(dummies, axis=1).drop(columns=['department', 'demographic', 'detail_type', 
                                                           'category', 'division', 'subcategory', 'type','extra_split'])



In [59]:
# Check the shape 
product_dummies.shape

(13732, 934)

In [60]:
# Check the last 5 rows of the DataFrame 
product_dummies.tail()

asin        color  \
17838  B00JBJ3HHO      missing   
17839  B00JGU2ZEE      missing   
17840  B00JKQR82S      missing   
17841  B00JR53ULU  unavailable   
17842  B00KA2X4QK      missing   

                                             description  details  \
17838  Seamless Communication: Samsung Gear 2 allows ...  missing   
17839  97% COTTON, 3% ELASTANE MACHINE WASHABLE SEE P...  missing   
17840  88% Cotton 12% Polyester Fine embroidery Slim ...  missing   
17841  Medium-size hoop earrings featuring three twis...  missing   
17842  No deductibles or added costs. Parts, labor an...  missing   

                                                    name     size  \
17838  Samsung Gear 2 Smartwatch - Metallic Orange (U...  missing   
17839  Lindy Bop 'Ophelia' Vintage 1950's Floral Spri...  missing   
17840  Rampage Womens Ultra Cute Embroidered Heather ...  missing   
17841  14k Gold-Bonded Sterling Silver Tri-Color Hoop...  missing   
17842  Kenneth Cole Reaction Easy To Remember, Black,...  missing   

       Arts_Crafts_Sewing  Automotive  Baby_Products  Beauty_Personal_Care  \
17838                   0           0              0                     0   
17839                   0           0              0                     0   
17840                   0           0              0                     0   
17841                   0           0              0                     0   
17842                   0           0              0                     0   

       ...  Wraps_Pashminas  Wrestling  Wrist_Watches  Y-Necklaces  \
17838  ...                0          0              0            0   
17839  ...                0          0              0            0   
17840  ...                0          0              0            0   
17841  ...                0          0              0            0   
17842  ...                0          0              0            0   

       Arm_Warmers  Baseball_Caps  Beanies__Knit_Hats  Berets  Briefs  Gloves  
17838            0              0                   0       0       0       0  
17839            0              0                   0       0       0       0  
17840            0              0                   0       0       0       0  
17841            0              0                   0       0       0       0  
17842            0              0                   0       0       0       0  

[5 rows x 934 columns]

In [61]:
# Merge reviews and product DataFrames on the asin column 
df = pd.merge(reviews, product_dummies, on='asin', how='left')

In [62]:
df.shape

(278677, 940)

In [64]:
#Checking if asin that were not able to be scraped have NaNs in the product info columns
df[df['asin'] == 'B007WA397M']

reviewerID        asin  \
196515  A1OFEEXTHPAROW  B007WA397M   
196516  A1C8L5G5NIWQZ8  B007WA397M   
196517  A33O1AU7FTFOEV  B007WA397M   
196518  A1EU9MOBPEHMUG  B007WA397M   
196519  A3VFQ0XO6TJ8BV  B007WA397M   
196520   AO4N0QPLSUHMQ  B007WA397M   
196521   ATHZKQ8IVYD61  B007WA397M   
196522   A1HLW9QWAEAT6  B007WA397M   
196523   AGQ52R6CDMO4R  B007WA397M   
196524   AUPTVTOI6SFKV  B007WA397M   
196525  A21XW1M1DOSJS5  B007WA397M   
196526  A22WP8TBN80WOB  B007WA397M   
196527  A33GU1NUNLEN47  B007WA397M   
196528   AKNDHLJEDY93U  B007WA397M   
196529   AASIQ2C5IV8XA  B007WA397M   
196530  A19LWHBTQJDZPC  B007WA397M   
196531   AYYJGPW5X90UE  B007WA397M   
196532   ABJ0WU4VEA3Z6  B007WA397M   
196533  A29NP54H9WI5TR  B007WA397M   

                                               reviewText  overall  \
196515  it's kinda small when i already ordered the ri...      2.0   
196516  I receive wonderful compliments when I wear th...      4.0   
196517  The color and shape is exactly the same as you...      4.0   
196518  I bought this from their website (and not from...      5.0   
196519  I placed like 4 orders so far for this seller....      2.0   
196520  I asked for extra small and they sent me a sma...      1.0   
196521  Allegra K Woman Stand Collar 3/4 Sleeve Flounc...      2.0   
196522  I got compliments on this all day... This shir...      5.0   
196523  I bought 10 items from Allegra K and this was ...      5.0   
196524  I love this shirt.bits nice enough to wear to ...      5.0   
196525  The fabric is kind of funny like paper but it ...      4.0   
196526  I paid attention to the sizing information and...      3.0   
196527  The fit is on the small size.  I will be givin...      3.0   
196528  The fabric is not flexible but is fair to the ...      3.0   
196529  I guess it should have been obvious from the p...      3.0   
196530  This blouse is the perfect size, very nice to ...      5.0   
196531  Alright for those who worry about SIZEI usuall...      3.0   
196532  So...this is an interesting shirt.For what I u...      3.0   
196533  The blouse is good for avg daily use.  The col...      4.0   

                                                  summary  unixReviewTime  \
196515                                              small      1395705600   
196516                                     Pretty in Pink      1394323200   
196517                                             Great.      1384905600   
196518                                               cute      1353888000   
196519             My last review and order for Allegra K      1350000000   
196520  extra small and they sent me a small size that...      1381622400   
196521                                      Not very good      1369699200   
196522                                 I LOVE THIS SHIRT!      1342483200   
196523                      Favorite shirt from Allegra K      1392681600   
196524                                     Adorable shirt      1386460800   
196525                                Runs small but cute      1390176000   
196526                                  Nice top but snug      1392854400   
196527                                    It's just okay.      1367971200   
196528                                          I like it      1380499200   
196529                  Too small and cheap construction.      1404172800   
196530                                          Very nice      1357171200   
196531                                 Sizing and Quality      1388016000   
196532                                      Works for Me!      1342483200   
196533                                       pretty color      1378857600   

       review_date color description details  ... Wraps_Pashminas Wrestling  \
196515  2014-03-25   NaN         NaN     NaN  ...             NaN       NaN   
196516  2014-03-09   NaN         NaN     NaN  ...             NaN       NaN   
196517  2013-11-20   NaN         NaN     NaN  ...             NaN       

In [65]:
#Check the percentage of NaN cells 
df.isnull().mean()

reviewerID                     0.000000
asin                           0.000000
reviewText                     0.000000
overall                        0.000000
summary                        0.000000
unixReviewTime                 0.000000
review_date                    0.000000
color                          0.393581
description                    0.393581
details                        0.393581
name                           0.393581
size                           0.393581
Arts_Crafts_Sewing             0.393581
Automotive                     0.393581
Baby_Products                  0.393581
Beauty_Personal_Care           0.393581
Cell_Phones_Accessories        0.393581
Clothing_Shoes_Jewelry         0.393581
Electronics                    0.393581
Health_Household               0.393581
Home_Kitchen                   0.393581
Industrial_Scientific          0.393581
Office_Products                0.393581
Purchase_Circles               0.393581
Software                       0.393581


In this case, Null values = products that couldn't be scraped.  Therefore, we will drop them.

In [66]:
df.dropna(inplace=True)

In [67]:
#Check for no null values 
df.isnull().sum().sum()

0

In [68]:
df[df['reviewerID'] == 'A1OFEEXTHPAROW']

reviewerID        asin  \
139799  A1OFEEXTHPAROW  B004VW55NA   
207441  A1OFEEXTHPAROW  B008AZYAJY   
235374  A1OFEEXTHPAROW  B00A76LPU6   

                                               reviewText  overall  \
139799  very poorly made,well,it was cheap.But I broug...      1.0   
207441  It fits very well!i like the color as well,plu...      5.0   
235374  i guess i shud have bought the next size,but t...      3.0   

                summary  unixReviewTime review_date        color  \
139799  i didnt like it      1390435200  2014-01-23  unavailable   
207441          luv it!      1376438400  2013-08-14      missing   
235374     little small      1402099200  2014-06-07      missing   

                                              description  details  ...  \
139799  Imported 20 millimeters wide Geneva Quality Go...        0  ...   
207441  Vintage Style Made from High Quality Materials...  missing  ...   
235374  Button closure International Size: XS, S, M; U...  missing  ...   

       Wraps_Pashminas Wrestling  Wrist_Watches  Y-Necklaces  Arm_Warmers  \
139799             0.0       0.0            0.0          0.0          0.0   
207441             0.0       0.0            0.0          0.0          0.0   
235374             0.0       0.0            0.0          0.0          0.0   

        Baseball_Caps  Beanies__Knit_Hats  Berets  Briefs  Gloves  
139799            0.0                 0.0     0.0     0.0     0.0  
207441            0.0                 0.0     0.0     0.0     0.0  
235374            0.0                 0.0     0.0     0.0     0.0  

[3 rows x 940 columns]

In [69]:
# Check first 5 rows to make sure everything's looking good.
df.head()

reviewerID        asin  \
0  A1KLRMWW2FWPL4  0000031887   
1  A2G5TCU2WDFZ65  0000031887   
2  A1RLQXYNCMWRWN  0000031887   
3   A8U3FAMSJVHS5  0000031887   
4  A3GEOILWLK86XM  0000031887   

                                          reviewText  overall  \
0  This is a great tutu and at a really great pri...      5.0   
1  I bought this for my 4 yr old daughter for dan...      5.0   
2  What can I say... my daughters have it in oran...      5.0   
3  We bought several tutus at once, and they are ...      5.0   
4  Thank you Halo Heaven great product for Little...      5.0   

                         summary  unixReviewTime review_date        color  \
0  Great tutu-  not cheaply made      1297468800  2011-02-12  unavailable   
1                    Very Cute!!      1358553600  2013-01-19  unavailable   
2       I have buy more than one      1357257600  2013-01-04  unavailable   
3               Adorable, Sturdy      1398556800  2014-04-27  unavailable   
4        Grammy's Angels Love it      1394841600  2014-03-15  unavailable   

                                         description details  ...  \
0  This fits your .     Make sure this fitsby ent...       0  ...   
1  This fits your .     Make sure this fitsby ent...       0  ...   
2  This fits your .     Make sure this fitsby ent...       0  ...   
3  This fits your .     Make sure this fitsby ent...       0  ...   
4  This fits your .     Make sure this fitsby ent...       0  ...   

  Wraps_Pashminas Wrestling  Wrist_Watches  Y-Necklaces  Arm_Warmers  \
0             0.0       0.0            0.0          0.0          0.0   
1             0.0       0.0            0.0          0.0          0.0   
2             0.0       0.0            0.0          0.0          0.0   
3             0.0       0.0            0.0          0.0          0.0   
4             0.0       0.0            0.0          0.0          0.0   

   Baseball_Caps  Beanies__Knit_Hats  Berets  Briefs  Gloves  
0            0.0                 0.0     0.0     0.0     0.0  
1            0.0                 0.0     0.0     0.0     0.0  
2            0.0                 0.0     0.0     0.0     0.0  
3            0.0                 0.0     0.0     0.0     0.0  
4            0.0                 0.0     0.0     0.0     0.0  

[5 rows x 940 columns]

In [70]:
# Check that values are all 1s and 0s 
df['Beanies__Knit_Hats'].value_counts()

0.0    168982
1.0        13
Name: Beanies__Knit_Hats, dtype: int64

In [71]:
# Check number of columns 
len(df.columns)

940

In [72]:
# Check number of unique columns 
len(set(df.columns))

823

We have duplicate columns names so we need to combine and drop.

In [73]:
# Make a list of columns names
list_cols = list(df.columns)
for item in set(df.columns):  # Loop through the set of unique column names 
    list_cols.remove(item)    # Remove items from full list of column names one time  

In [74]:
len(list_cols)  # Leaves us with only the 118 duplicated column names 

117

In [75]:
# Check the first 20 
list_cols[:20]

['Baby',
 'Belts',
 'Boys',
 'Girls',
 'Gloves_Mittens',
 'Hats_Caps',
 'Leggings',
 'Men',
 'Pants',
 'Scarves',
 'Shirts',
 'Shorts',
 'Skirts',
 'Skirts_Skorts',
 'Socks',
 'Suspenders',
 'T-Shirts',
 'Women',
 'Accessories',
 'Backpacks']

Created a for loop to check every item in `list_cols` and rename it and keep it's highest value of 1 or 0.

In [76]:
for c in list_cols:  # For every column name in the list of column names 
    df[c + "_agg"] = df[c].max(axis=1)  # Rename column with column name + _agg and keep the max value for that row
  

In [77]:
len(df.columns) # Check new length of columns 

1038

Check an example to make sure it worked.

In [78]:
# Check that all the Backpack columns combined but only kept 1s and 0s 
df['Backpacks_agg'].value_counts()

0.0    168243
1.0       752
Name: Backpacks_agg, dtype: int64

Drop all columns from DataFrame that are in list_cols 

In [79]:
df.drop(columns=list_cols, inplace=True)

In [1]:
Check that there are not more duplicated columns 

SyntaxError: invalid syntax (<ipython-input-1-af63c3af123c>, line 1)

In [80]:
df.columns.duplicated().sum()

0

In [81]:
df.shape

(168995, 823)

In [82]:
df.head()

reviewerID        asin  \
0  A1KLRMWW2FWPL4  0000031887   
1  A2G5TCU2WDFZ65  0000031887   
2  A1RLQXYNCMWRWN  0000031887   
3   A8U3FAMSJVHS5  0000031887   
4  A3GEOILWLK86XM  0000031887   

                                          reviewText  overall  \
0  This is a great tutu and at a really great pri...      5.0   
1  I bought this for my 4 yr old daughter for dan...      5.0   
2  What can I say... my daughters have it in oran...      5.0   
3  We bought several tutus at once, and they are ...      5.0   
4  Thank you Halo Heaven great product for Little...      5.0   

                         summary  unixReviewTime review_date        color  \
0  Great tutu-  not cheaply made      1297468800  2011-02-12  unavailable   
1                    Very Cute!!      1358553600  2013-01-19  unavailable   
2       I have buy more than one      1357257600  2013-01-04  unavailable   
3               Adorable, Sturdy      1398556800  2014-04-27  unavailable   
4        Grammy's Angels Love it      1394841600  2014-03-15  unavailable   

                                         description details  ...  \
0  This fits your .     Make sure this fitsby ent...       0  ...   
1  This fits your .     Make sure this fitsby ent...       0  ...   
2  This fits your .     Make sure this fitsby ent...       0  ...   
3  This fits your .     Make sure this fitsby ent...       0  ...   
4  This fits your .     Make sure this fitsby ent...       0  ...   

  Tops_Tees_agg Trunks_agg  Umbrellas_agg  Underwear_agg  Wallets_agg  \
0           0.0        0.0            0.0            0.0          0.0   
1           0.0        0.0            0.0            0.0          0.0   
2           0.0        0.0            0.0            0.0          0.0   
3           0.0        0.0            0.0            0.0          0.0   
4           0.0        0.0            0.0            0.0          0.0   

   Wear_to_Work_agg  Wrist_Watches_agg  Arm_Warmers_agg  Baseball_Caps_agg  \
0               0.0                0.0              0.0                0.0   
1               0.0                0.0              0.0                0.0   
2               0.0                0.0              0.0                0.0   
3               0.0                0.0              0.0                0.0   
4               0.0                0.0              0.0                0.0   

   Berets_agg  
0         0.0  
1         0.0  
2         0.0  
3         0.0  
4         0.0  

[5 rows x 823 columns]

Rename columns with _agg suffix 

In [83]:
# Look at the first 5 rows for the new _agg columns 
df.iloc[:5 , -98:]

Baby_agg  Belts_agg  Boys_agg  Girls_agg  Gloves_Mittens_agg  \
0       0.0        0.0       0.0        0.0                 0.0   
1       0.0        0.0       0.0        0.0                 0.0   
2       0.0        0.0       0.0        0.0                 0.0   
3       0.0        0.0       0.0        0.0                 0.0   
4       0.0        0.0       0.0        0.0                 0.0   

   Hats_Caps_agg  Leggings_agg  Men_agg  Pants_agg  Scarves_agg  ...  \
0            0.0           0.0      0.0        0.0          0.0  ...   
1            0.0           0.0      0.0        0.0          0.0  ...   
2            0.0           0.0      0.0        0.0          0.0  ...   
3            0.0           0.0      0.0        0.0          0.0  ...   
4            0.0           0.0      0.0        0.0          0.0  ...   

   Tops_Tees_agg  Trunks_agg  Umbrellas_agg  Underwear_agg  Wallets_agg  \
0            0.0         0.0            0.0            0.0          0.0   
1            0.0         0.0            0.0            0.0          0.0   
2            0.0         0.0            0.0            0.0          0.0   
3            0.0         0.0            0.0            0.0          0.0   
4            0.0         0.0            0.0            0.0          0.0   

   Wear_to_Work_agg  Wrist_Watches_agg  Arm_Warmers_agg  Baseball_Caps_agg  \
0               0.0                0.0              0.0                0.0   
1               0.0                0.0              0.0                0.0   
2               0.0                0.0              0.0                0.0   
3               0.0                0.0              0.0                0.0   
4               0.0                0.0              0.0                0.0   

   Berets_agg  
0         0.0  
1         0.0  
2         0.0  
3         0.0  
4         0.0  

[5 rows x 98 columns]

In [84]:
# Create a dataframe with just the columns I want to rename
df_agg = df.iloc[: , -98:]

In [85]:
rename_dict ={} # empty dictionary 
for c in df.columns:  # loop through every column name in the dataframe 
    if c in list(df_agg.columns): # if the column name is in the list of column names that end with _agg 
        rename_dict[c] = c[:-4]   # slice off the last 4 characters of the column name and add it to the rename dict
df.rename(columns=rename_dict, inplace=True)    # rename the columns with the new names in the dictionary

#Check that it worked
df.iloc[:10 , -98:]

Baby  Belts  Boys  Girls  Gloves_Mittens  Hats_Caps  Leggings  Men  Pants  \
0   0.0    0.0   0.0    0.0             0.0        0.0       0.0  0.0    0.0   
1   0.0    0.0   0.0    0.0             0.0        0.0       0.0  0.0    0.0   
2   0.0    0.0   0.0    0.0             0.0        0.0       0.0  0.0    0.0   
3   0.0    0.0   0.0    0.0             0.0        0.0       0.0  0.0    0.0   
4   0.0    0.0   0.0    0.0             0.0        0.0       0.0  0.0    0.0   
5   0.0    0.0   0.0    0.0             0.0        0.0       0.0  0.0    0.0   
6   0.0    0.0   0.0    0.0             0.0        0.0       0.0  0.0    0.0   
7   0.0    0.0   0.0    0.0             0.0        0.0       0.0  0.0    0.0   
8   0.0    0.0   0.0    0.0             0.0        0.0       0.0  0.0    0.0   
9   0.0    0.0   0.0    0.0             0.0        0.0       0.0  0.0    0.0   

   Scarves  ...  Tops_Tees  Trunks  Umbrellas  Underwear  Wallets  \
0      0.0  ...        0.0     0.0        0.0        0.0      0.0   
1      0.0  ...        0.0     0.0        0.0        0.0      0.0   
2      0.0  ...        0.0     0.0        0.0        0.0      0.0   
3      0.0  ...        0.0     0.0        0.0        0.0      0.0   
4      0.0  ...        0.0     0.0        0.0        0.0      0.0   
5      0.0  ...        0.0     0.0        0.0        0.0      0.0   
6      0.0  ...        0.0     0.0        0.0        0.0      0.0   
7      0.0  ...        0.0     0.0        0.0        0.0      0.0   
8      0.0  ...        0.0     0.0        0.0        0.0      0.0   
9      0.0  ...        0.0     0.0        0.0        0.0      0.0   

   Wear_to_Work  Wrist_Watches  Arm_Warmers  Baseball_Caps  Berets  
0           0.0            0.0          0.0            0.0     0.0  
1           0.0            0.0          0.0            0.0     0.0  
2           0.0            0.0          0.0            0.0     0.0  
3           0.0            0.0          0.0            0.0     0.0  
4           0.0            0.0          0.0            0.0     0.0  
5           0.0            0.0          0.0            0.0     0.0  
6           0.0            0.0          0.0            0.0     0.0  
7           0.0            0.0          0.0            0.0     0.0  
8           0.0            0.0          0.0            0.0     0.0  
9           0.0            0.0          0.0            0.0     0.0  

[10 rows x 98 columns]

In [86]:
df.columns 

Index(['reviewerID', 'asin', 'reviewText', 'overall', 'summary',
       'unixReviewTime', 'review_date', 'color', 'description', 'details',
       ...
       'Tops_Tees', 'Trunks', 'Umbrellas', 'Underwear', 'Wallets',
       'Wear_to_Work', 'Wrist_Watches', 'Arm_Warmers', 'Baseball_Caps',
       'Berets'],
      dtype='object', length=823)

Save work so far to a csv in the data folder.

In [85]:
# df.to_csv('data/reviews_and_products.csv')

In [88]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [89]:
df['boys'].hist()

KeyError: 'boys'